This notebook attempts to use The RCA framework to do QC on image segmentation, but using the exact same CNN used for doing the segmentation as the starting point.

In [ ]:
from tensorflow.keras.models import model_from_json,clone_model
from tensorflow.keras.optimizers import Adam

from mask_utils import iou

from network_utils import augmentImageSequence

import numpy as np

import os

# from sklearn.model_selection import train_test_split

import copy

import matplotlib.pyplot as plt

Load data and model, and get them sorted in the same way as in the notebooks used to train models (i.e. same train/test split, random seed etc)

In [ ]:
DataDir = './data/pericardial/wsx_round2/'

#load data - these files created by extract_dcm_for_wsx.ipynb
X = np.load(os.path.join(DataDir,'X.npy'))
Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')
pxArea = np.load(os.path.join(DataDir,'pxSize.npy'))
pxSpacing = np.sqrt(pxArea)

#ensure the shape is correct arrays saved were rank 3, so this changes to rank 4 (last dimension represents channels)
X = X.reshape([*X.shape,1])
Y = Y.reshape([*Y.shape,1])

#do train/test split!
# X_train, X_test, Y_train, Y_test,pxArea_train,pxArea_test,pxSpacing_train,pxSpacing_test = train_test_split(X, Y, pxArea,pxSpacing, test_size=0.2,random_state=101)

X = X[:100,:,:,:]
Y = Y[:100,:,:,:]

#
# M = X.shape[0]
# MTest = X_test.shape[0]

Now, need to load a model which can be used for the RCA

In [ ]:
#pick a model, just need one to play with.
modelBaseName = './data/models/mrunet_2020-04-07_09:59' #THIS MODEL IS NOT A GOOD ONE BUT HAS BEEN SELECTED TO GIVE A WIDE SPREAD IN IOU ON TRAIN AND TEST SETS

#load the model archistecture
with open( modelBaseName + '.json', 'r') as json_file:
    model = model_from_json( json_file.read() )
    
#get the weights
model.load_weights(modelBaseName + '.h5')    

NOTE! RCA uses a network output to train a classifier which can be applied to other labelled data. I plan to use as a starting point *the actual network* that was used to generate the segmentation. However, this will not work in the obvious way - training a network on its own output will result in the cost function ==0, and thus all gradients ==0. 

However, thresholding the network output first will work! as the network will never output 0/1, but some numbers close to them.

In [ ]:
OPT = Adam(learning_rate = 1e-3,
           beta_1 = 0.9,
           beta_2 = 0.999,
           amsgrad = False
          )


def retrain_model(model,x,y):#,optimizer):
    
    ''''''
    
    assert np.all(model.input_shape[1:] == x.shape[1:]) and np.all(model.input_shape[1:] == y.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    
    assert x.shape[0] == 1 and y.shape[0]==1, 'you can only do RCA on one image at a time!'
    
    #threshold mask so that it can be used as a target for CNN
    y = y > 0.5
    
    #make a complete local copy of the model so that it is not modified globally
    weights = model.get_weights()
    model_local = clone_model(model)
    model_local.set_weights(weights)
    
    model_local.compile(optimizer = OPT, 
                        loss = 'binary_crossentropy'
                       )
    
    model_local.fit(x=x,
                    y=y,
                    epochs = 10, #THINK ABOUT ME
                    steps_per_epoch= 1, #obvs
                    verbose=0
                   )
    
    return model_local

In [ ]:
def evaluate_model(model,X_val,Y_val):
    
    '''this function takes a model (presumably retrained on a predicted mask in order to do RCA) and evaluates it on the set of masks which are known'''

#     assert np.all(X.shape==Y.shape),'looks like you have mismatched your images and masks'
#     assert X.shape[0]>1,'you should only use this on more than one image. Are you doing what you think youre doing?
    
    
    Y_pred = model.predict(X_val)
    
    ious = np.array([iou(Y_pred[m],Y_val[m]) for m in range(Y_pred.shape[0])])
    
    return ious

In [ ]:
def predict_and_RCA_evaluate(model,x,X_val,Y_val):

    assert np.all(model.input_shape[1:] == x.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    assert x.shape[0] == 1, 'you can only do RCA on one image at a time!'
    
    y = model.predict(x)
    
    rca_model = retrain_model(model,x,y)
    
    ious = evaluate_model(rca_model,X_val,Y_val)
    
    return ious

First, we should look at the iou spread over the whole thing...

In [ ]:
allious = evaluate_model(model,X,Y)

plt.hist(allious,bins = np.arange(0,1.05,0.05))

Now, lets just get a single datapoint to play with..`

In [ ]:
#select an example image
# np.random.seed(7)
egInd = np.random.randint(X.shape[0])

#get the IOU that we want to predict...
trueIOU = allious[egInd]

#get the actual image out and shaped correctly
egX = X[egInd,:,:].reshape(1,*model.input_shape[1:])

#get all images EXCEPT that one, from both X and Y
mask = np.ones(X.shape[0],dtype=bool)
mask[egInd] = False
X_val = X[mask,:,:,:]
Y_val = Y[mask,:,:,:]

predictedIOUs = predict_and_RCA_evaluate(model,egX,X_val,Y_val)

plt.hist(predictedIOUs,bins= np.arange(0,1.05,0.05),density=True)

plt.plot([trueIOU,trueIOU],plt.ylim(),c='r')


In [ ]:
def get_predicted_ious(ind):
    
    #get the actual image out and shaped correctly
    egX = X[egInd,:,:].reshape(1,*model.input_shape[1:])

    #get all images EXCEPT that one, from both X and Y
    mask = np.ones(X.shape[0],dtype=bool)
    mask[egInd] = False
    X_val = X[mask,:,:,:]
    Y_val = Y[mask,:,:,:]

    predictedIOUs = predict_and_RCA_evaluate(model,egX,X_val,Y_val)
    
    return predictedIOUs.reshape(1,-1)


predIOUs = np.concatenate([get_predicted_ious(ind) for ind in range(X.shape[0])],axis=0)

In [ ]:
from scipy.stats import pearsonr

In [ ]:
plt.figure(figsize = (5,15))

y = np.max(predIOUs,axis=1)
plt.subplot(3,1,1)
plt.plot([0,0],[1,1])
plt.scatter(allious,y)
plt.title(f'{pearsonr(allious,y)[0]**2:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (max)')                      

y = np.median(predIOUs,axis=1)            
plt.subplot(3,1,2)
plt.plot([0,0],[1,1])
plt.scatter(allious,y)
plt.title(f'{pearsonr(allious,y)[0]**2:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (median)')                      
  
y = np.mean(predIOUs,axis=1)
plt.subplot(3,1,3)
plt.plot([0,0],[1,1])
plt.scatter(allious,y)
plt.title(f'{pearsonr(allious,y)[0]**2:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (mean)')                      

            
plt.xlabel('true IOU')
